In [1]:
#Packages

import json
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, StaleElementReferenceException
from selenium import webdriver
import os
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
import requests
from datetime import datetime, timedelta

# Go get geckodriver from : https://github.com/mozilla/geckodriver/releases

In [2]:
def ffx_preferences(dfolder, download=False):
    '''
    Sets the preferences of the firefox browser: download path.
    '''
    profile = webdriver.FirefoxProfile()
    # set download folder:
    profile.set_preference("browser.download.dir", dfolder)
    profile.set_preference("browser.download.folderList", 2)
    profile.set_preference("browser.download.manager.showWhenStarting", False)
    profile.set_preference("browser.helperApps.neverAsk.saveToDisk",
                           "application/msword,application/rtf, application/csv,text/csv,image/png ,image/jpeg, application/pdf, text/html,text/plain,application/octet-stream")
    
   # profile.add_extension('/Users/luisignaciomenendezgarcia/Dropbox/CLASSES/class_bse_text_mining/class_scraping_bse/booking/booking/ublock_origin-1.55.0.xpi')


    # this allows to download pdfs automatically
    if download:
        profile.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/pdf,application/x-pdf")
        profile.set_preference("pdfjs.disabled", True)

    options = Options()
    options.profile = profile
    return options


def start_up(link, dfolder, geko_path,donwload=True):
    # geko_path='/Users/luisignaciomenendezgarcia/Dropbox/CLASSES/class_bse_text_mining/class_scraping_bse/booking/geckodriver'
    # download_path='./downloads'
    os.makedirs(dfolder, exist_ok=True)

    options = ffx_preferences(dfolder,donwload)
    service = Service(geko_path)
    browser = webdriver.Firefox(service=service, options=options)
    # Enter the website address here
    browser.get(link)
    time.sleep(5)  # Adjust sleep time as needed
    return browser


def check_and_click(browser, xpath, type):
    '''
    Function that checks whether the object is clickable and, if so, clicks on
    it. If not, waits one second and tries again.
    '''
    ck = False
    ss = 0
    while ck == False:
        ck = check_obscures(browser, xpath, type)
        time.sleep(1)
        ss += 1
        if ss == 15:
            # warn_sound()
            # return NoSuchElementException
            ck = True
            # browser.quit()

def check_obscures(browser, xpath, type):
    '''
    Function that checks whether the object is being "obscured" by any element so
    that it is not clickable. Important: if True, the object is going to be clicked!
    '''
    try:
        if type == "xpath":
            browser.find_element('xpath',xpath).click()
        elif type == "id":
            browser.find_element('id',xpath).click()
        elif type == "css":
            browser.find_element('css selector',xpath).click()
        elif type == "class":
            browser.find_element('class name',xpath).click()
        elif type == "link":
            browser.find_element('link text',xpath).click()
    except (ElementClickInterceptedException, NoSuchElementException, StaleElementReferenceException) as e:
        print(e)
        return False
    return True

## Opening Booking.com

In [3]:
# lets open booking:

dfolder='./downloads'
geko_path='/Users/mikelgallo/repos2/text_mining/Scraping_UN/books/geckodriver'
link='https://www.booking.com/index.es.html'

browser=start_up(dfolder=dfolder,link=link,geko_path=geko_path)

#### Pending - Removing popups

#### Opening Search_box

In [4]:
browser.find_element(by='xpath',value= '//input[@id=":re:"]').click()

#### Inputing destination

In [5]:
place = input('Where do you want to go?')
search1 = browser.find_element(by='xpath',value='//*[@id=":re:"]')
search1.send_keys(place)

Where do you want to go? Barcelona


#### Opening Dates table

In [6]:
#Use CSS Selector Copy
css='button.ebbedaf8ac:nth-child(2) > span:nth-child(1)'
browser.find_element('css selector',css).click()

#### Selecting Dates

In [8]:
#Selecting Dates table and retrieving date information
def extract_dates():
    dates_f = []
    path='//div[@id="calendar-searchboxdatepicker"]//table[@class="eb03f3f27f"]//tbody//td[@class="b80d5adb18"]//span[@class="cf06f772fa"]'
    dates = browser.find_elements('xpath',path)
    for date in dates:
        dates_f.append(date)
    return dates_f

In [9]:
dates_f = extract_dates()

for date in dates_f:
    print(date.get_attribute("data-date"))

2024-01-22
2024-01-23
2024-01-24
2024-01-25
2024-01-26
2024-01-27
2024-01-28
2024-01-29
2024-01-30
2024-01-31
2024-02-01
2024-02-02
2024-02-03
2024-02-04
2024-02-05
2024-02-06
2024-02-07
2024-02-08
2024-02-09
2024-02-10
2024-02-11
2024-02-12
2024-02-13
2024-02-14
2024-02-15
2024-02-16
2024-02-17
2024-02-18
2024-02-19
2024-02-20
2024-02-21
2024-02-22
2024-02-23
2024-02-24
2024-02-25
2024-02-26
2024-02-27
2024-02-28
2024-02-29


In [10]:
for date in dates_f:
    if date.get_attribute("data-date") == "2024-01-22":
        time.sleep(1)
        date.click()
    if date.get_attribute("data-date") == "2024-01-27":
        time.sleep(1)
        date.click()
        break

In [11]:
#Returning pages
#for i in range(1,6):
    #if i == 1:
        #x_path = '/html/body/div[3]/div[2]/div/form/div[1]/div[2]/div/div[2]/div/nav/div[2]/div/div[1]/button[1]'
    #else:
        #x_path = '/html/body/div[3]/div[2]/div/form/div[1]/div[2]/div/div[2]/div/nav/div[2]/div/div[1]/button[1]'
    #browser.find_element(by='xpath',value=x_path).click()
    #time.sleep(2)

#### Inputing Dates 

### Searching

In [11]:
my_xpath='/html/body/div[3]/div[2]/div/form/div[1]/div[4]/button/span'

check_obscures(browser,my_xpath , type='xpath')
check_and_click(browser,my_xpath , type='xpath')

Message: Unable to locate element: /html/body/div[3]/div[2]/div/form/div[1]/div[4]/button/span; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:189:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:507:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:132:16

Message: Unable to locate element: /html/body/div[3]/div[2]/div/form/div[1]/div[4]/button/span; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:189:5
NoSuchElementError@chrome://remote/content/shared/webdriver/E

#### Retrieve data

### Getting the number of Pages

In [13]:

def get_number_pages(browser):
    '''
    Get the number of pages. 
    '''
    a = browser.find_elements('xpath','//div[@data-testid="pagination"]//li[@class="b16a89683f"]')
    return(int(a[-1].text))


pages = get_number_pages(browser)

print(pages)

32


#### Creating Click button

In [14]:
#Click next page
x_path='/html/body/div[4]/div/div[2]/div/div[2]/div[3]/div[2]/div[2]/div[4]/div[2]/nav/nav/div/div[3]/button/span/span'
#browser.find_element(by='xpath',value=x_path).click()

#### Dates to look for

In [21]:
link = browser.find_elements('xpath',"//div[@class='d6767e681c']//a")
links = []
for l in link:
    #print(l.get_attribute('href'))
    links.append(l.get_attribute('href'))
for l in links:
    print(l)

https://www.booking.com/hotel/es/h10-casa-mimosa.es.html?label=gen173nr-1FCAEoggI46AdIClgEaEaIAQGYAQq4AQfIAQ_YAQHoAQH4AQKIAgGoAgO4AvK4ua0GwAIB0gIkMjBlNmQ2OTUtMGY4Zi00Njc3LWEwZTQtN2M1YjU0MmFlZmQ12AIF4AIB&aid=304142&ucfs=1&arphpl=1&checkin=2024-01-22&checkout=2024-01-27&dest_id=-372490&dest_type=city&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=1&hapos=1&sr_order=popularity&srpvid=3d8f568002c40332&srepoch=1705925892&all_sr_blocks=186920103_242553103_2_2_0&highlighted_blocks=186920103_242553103_2_2_0&matching_block_id=186920103_242553103_2_2_0&sr_pri_blocks=186920103_242553103_2_2_0__115570&from_sustainable_property_sr=1&from=searchresults#hotelTmpl
https://www.booking.com/hotel/es/casa-luz-barcelona.es.html?label=gen173nr-1FCAEoggI46AdIClgEaEaIAQGYAQq4AQfIAQ_YAQHoAQH4AQKIAgGoAgO4AvK4ua0GwAIB0gIkMjBlNmQ2OTUtMGY4Zi00Njc3LWEwZTQtN2M1YjU0MmFlZmQ12AIF4AIB&aid=304142&ucfs=1&arphpl=1&checkin=2024-01-22&checkout=2024-01-27&dest_id=-372490&dest_type=city&group_adult

In [33]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

num_pages = 0
df = []

for i in range(1, 4):
    x_path = '/html/body/div[4]/div/div[2]/div/div[2]/div[3]/div[2]/div[2]/div[4]/div[2]/nav/nav/div/div[3]/button/span/span'
    
    hotels = browser.find_elements('xpath', '//div[@class="f6431b446c a15b38c233"]')
    ratings = browser.find_elements('xpath', '//div[@class="a3b8729ab1 d86cee9b25"]')
    prices = browser.find_elements('xpath', '//span[@class="f6431b446c fbfd7c1165 e84eb96b1f"]')
    districts_list = []

    districts = browser.find_elements('xpath', '//span[@class="aee5343fdb def9bc142a"]')
    for i in districts:
        if ', barcelona' in i.text.lower():
            districts_list.append(i)

    # Retrieving only center distances
    distance_center = []
    distance = browser.find_elements('xpath', '//div[@class="abf093bdfe ecc6a9ed89"]//span[@class="f419a93f12"]')
    for i in distance:
        if 'centro' in i.text:
            distance_center.append(i)

    for a, b, c, d, e in zip(hotels, ratings, distance_center, districts_list, prices):
        try:
            row_data = {'Hotels': a.text, 'Ratings': b.text, 'Distance': c.text, 'District': d.text, 'Price': e.text}
            print(row_data)
            df.append(row_data)
        except Exception as e:
            row_none = {'Hotels': None, 'Ratings': None, 'Distance': None, 'District': None, 'Price': None}
            df.append(row_none)
            print(row_none)

    # Adding explicit wait after clicking the "Next" button
    wait = WebDriverWait(browser, 10)  # Adjust the timeout as needed
    next_button = wait.until(EC.element_to_be_clickable((By.XPATH, x_path)))
    next_button.click()
    num_pages += 1


{'Hotels': 'H10 Casa Mimosa 4* Sup', 'Ratings': '9,2', 'Distance': 'a 1,2 km del centro', 'District': 'Eixample, Barcelona', 'Price': '€ 946'}
{'Hotels': 'Sonder Casa Luz', 'Ratings': '8,4', 'Distance': 'a 450 m del centro', 'District': 'Eixample, Barcelona', 'Price': '€ 665'}
{'Hotels': 'StayCenter - Gothic Apartments & Studios', 'Ratings': '7,6', 'Distance': 'a 1,1 km del centro', 'District': 'Ciutat Vella, Barcelona', 'Price': '€ 474'}
{'Hotels': 'H La Paloma Adults Only', 'Ratings': '7,9', 'Distance': 'a 0,7 km del centro', 'District': 'Ciutat Vella, Barcelona', 'Price': '€ 634'}
{'Hotels': 'BARCELONA GOTIC Guesthouse', 'Ratings': '8,3', 'Distance': 'a 1 km del centro', 'District': 'Ciutat Vella, Barcelona', 'Price': '€ 200'}
{'Hotels': 'Ilunion Barcelona', 'Ratings': '8,6', 'Distance': 'a 3 km del centro', 'District': 'Sant Martí, Barcelona', 'Price': '€ 403'}
{'Hotels': 'Hostal Orleans', 'Ratings': '7,3', 'Distance': 'a 1,2 km del centro', 'District': 'Ciutat Vella, Barcelona', '

In [34]:
df_new = pd.DataFrame(df)
df_new.shape

(75, 5)

In [35]:
df_new[0:50]

,Hotels,Ratings,Distance,District,Price
0,H10 Casa Mimosa 4* Sup,"9,2","a 1,2 km del centro","Eixample, Barcelona",€ 946
1,Sonder Casa Luz,"8,4",a 450 m del centro,"Eixample, Barcelona",€ 665
2,StayCenter - Gothic Apartments & Studios,"7,6","a 1,1 km del centro","Ciutat Vella, Barcelona",€ 474
3,H La Paloma Adults Only,"7,9","a 0,7 km del centro","Ciutat Vella, Barcelona",€ 634
4,BARCELONA GOTIC Guesthouse,"8,3",a 1 km del centro,"Ciutat Vella, Barcelona",€ 200
5,Ilunion Barcelona,"8,6",a 3 km del centro,"Sant Martí, Barcelona",€ 403
6,Hostal Orleans,"7,3","a 1,2 km del centro","Ciutat Vella, Barcelona",€ 456
7,Travelodge Barcelona Poblenou,"7,5","a 2,9 km del centro","Sant Martí, Barcelona",€ 335
8,Hotel Best Front Maritim,"8,7","a 4,1 km del centro","Sant Martí, Barcelona",€ 568
9,Hotel Paxton Barcelona,"7,4","a 2,9 km del centro","Sant Martí, Barcelona",€ 614


In [22]:
browser.quit()